In [60]:
import numpy as np
import ast
import pytz
import math
import itertools
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_selection import r_regression 

df = pd.read_csv('top10K-spotify.csv',index_col=0)
df["type"].value_counts()
df=df.drop(columns=["index","city_1","district_1","district_2","city_3","district_3" ])
df = df[df['city_2'].notna()]
df = df[df['country'].notna()]
df.rename(columns = {'artist':'name'}, inplace = True)

df2=pd.read_csv('data_w_spotify.csv',index_col=None)
df2=df2.drop(columns=["isdone","id"])
df2['genres'] = df2['genres'].fillna('')

df3=df.merge(df2,on='name')

df3['genres']= df3['genres'].str.strip('[]').str.replace("'","").str.split('\s*,\s*')

def create_soup(x):
    return ' '.join(x['genres']) + ' '+ x['city_2'] + ' ' + x['country']+' ' + x['gender']+ ' ' + x['type']

df3['soup'] = df3.apply(create_soup, axis=1)
print(df3.shape,df3.columns)

dict_names = dict(zip(df3.name, df3.spotifyid))

df = df.reset_index()
indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()



def get_recommendations(name,sim,reverse):
    print(sim[0:20])
    # print(name,sim)
    indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()
    idx = indices[name]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=reverse)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

     # Get the movie indices
    indices = [i[0] for i in sim_scores]
    scores = [round(i[1]*100, 2) for i in sim_scores]

    # Return the top 10 most similar movies
     
    top_10=df3['name'].iloc[indices]
    spot_top10=[]
    for item in top_10:
        spot_top10.append(dict_names[str(item)])

    artist=dict_names[str(df3['name'].iloc[idx])]
    
    wantedDict = {
        'mainId': artist,
        'top10': spot_top10,
        'scores': scores
    }

    return top_10




(1416, 11) Index(['name', 'gender', 'age', 'type', 'country', 'city_2', 'spotifyid',
       'genres', 'popularity', 'followers', 'soup'],
      dtype='object')


In [57]:
def CVectorizer():
    count = CountVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def CosSim(x):
    return (cosine_similarity(x,x),True)

In [58]:
def TfidVectorizer():
    count = TfidfVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def EuDSim(x):
    return (euclidean_distances(x,), False)

In [61]:
#matrix=TfidVectorizer()
matrix=CVectorizer()
sim,reverse = EuDSim(matrix)
#im,reverse = CosSim(matrix)

get_recommendations("Drake",sim,reverse)

[[0.         1.27164672 1.37760589 ... 1.40778399 1.40174478 1.39498871]
 [1.27164672 0.         1.39319914 ... 1.41421356 1.39903586 1.40314814]
 [1.37760589 1.39319914 0.         ... 1.39020015 1.39098547 1.35909005]
 ...
 [1.37657832 1.40591266 0.99703375 ... 1.37564559 1.40504265 1.34643161]
 [1.29901872 1.06696599 1.38947701 ... 1.41421356 1.39635422 1.40119646]
 [1.38786089 1.39470118 1.32327071 ... 1.38721054 1.40777225 1.36691081]]


134        Tory Lanez
191      Jessie Reyez
306           JP Saxe
21       Shawn Mendes
795              dvsn
299               Kai
300               Kai
382      Majid Jordan
1029       Dragonette
157     PARTYNEXTDOOR
Name: name, dtype: object

In [ ]:
tfid_matrix=TfidVectorizer()
cos_sim=CosSim(CV_matrix)

get_recommendations("Drake",cos_sim)